In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d omkargurav/face-mask-dataset

In [ ]:
from zipfile import ZipFile
dataset = '/content/face-mask-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The Dataset is Extracted')

**Importing the Libraries**

In [ ]:
import os # to access the file in folder
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
with_mask_files = os.listdir('/content/data/with_mask')
print(with_mask_files[0:5])
print(with_mask_files[-5:])

In [ ]:
without_mask_files = os.listdir('/content/data/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

In [ ]:
print('Number of with mask images:', len(with_mask_files))
print('Number of without mask images:', len(without_mask_files))

**Creating Labels for the two class of Images**

with mask ---> 1

without mask ---> 0

In [ ]:
with_mask_labels = [1]*3725
without_mask_labels = [0]*3828

In [ ]:
print(with_mask_labels[0:5])
print(without_mask_labels[-5:])

In [ ]:
print(len(with_mask_labels))
print(len(without_mask_labels))

In [ ]:
labels = with_mask_labels + without_mask_labels
print(len(labels))

**Displaying the Images**

In [ ]:
img = mpimg.imread('/content/data/with_mask/with_mask_2619.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img = mpimg.imread('/content/data/without_mask/without_mask_1307.jpg')
imgplot = plt.imshow(img)
plt.show()

**Image Processing**

In [ ]:
with_mask_path = '/content/data/with_mask/'
data = []
for img_file in with_mask_files:
  image = Image.open(with_mask_path + img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)



without_mask_path = '/content/data/without_mask/'
for img_file in without_mask_files:
  image = Image.open(without_mask_path + img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)

In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
data[0]

In [ ]:
data[0].shape

In [ ]:
X = np.array(data)
y = np.array(labels)


In [ ]:
type(X)

In [ ]:
print(X.shape)
print(y.shape)

**Train Test Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape )

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
X_train_scaled[0]

**Building a Convolution Neural Network**

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
num_of_classes =2
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size = (3,3), activation= 'relu', input_shape = (128, 128, 3)))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2)))


model.add(keras.layers.Conv2D(64, kernel_size = (3,3), activation= 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2)))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = 'relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(64, activation = 'relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(num_of_classes, activation = 'sigmoid'))


In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics =  ['acc'])

In [ ]:
history = model.fit(X_train_scaled, y_train, validation_split = 0.1, epochs = 5)

**Model Evaluation**

In [ ]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print('Test Accuravy =', accuracy)

In [ ]:
h= history
plt.plot(h.history['loss'], label = 'train loss')
plt.plot(h.history['val_loss'], label = 'validation loss')
plt.legend()
plt.show()


In [ ]:
plt.plot(h.history['acc'], label = 'train accuracy')
plt.plot(h.history['val_acc'], label = 'validation accuracy')
plt.legend()
plt.show()

**Predictive System**

In [ ]:
input_image_path = input('Path of the image to be predictede: ')
input_image = cv2.imread(input_image_path)
cv2_imshow(input_image)
input_image_resized = cv2.resize(input_image, (128,128))
input_image_scaled = input_image_resized/255
input_image_reshaped = np.reshape(input_image_scaled, [1,128,128,3])
input_prediction = model.predict(input_image_reshaped)
print(input_prediction)
input_pred_label = np.argmax(input_prediction)
print(input_pred_label)

if input_pred_label == 1:
 print('The person in the image is wearing mask')

else:
  print('The person in the image is not wearing mask')
